In [30]:
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np

In [32]:
class perceptron_layer:

    def __init__(self, no_input, no_output, lr, act_func, act_deriv_given_act):
        self.lr = lr
        self.params = np.random.rand(no_output, no_input + 1)
        self.activation_function = act_func
        self.activation_derivative_given_activation = act_deriv_given_act
        print(self)
        
    def evaluate(self, inputs):
        #return self.params * inputs.append(1)
        inputs = np.append(inputs, [1]).T
        self.activations = inputs
        output = self.activation_function(np.dot(self.params, inputs))
        return output
    
    def __repr__(self):
        return repr(self.params)
    
    def training_forward(self, inputs):
        inputs = np.append(inputs, [1]).T
        self.activations = inputs
        self.z = z = np.dot(self.params, inputs)
        self.a = output = self.activation_function(z)
        return output
    
    def error_previous_layer(self, delta_l_plus_1):
        #δl=((wl+1)Tδl+1)⊙σ′(zl)
        inputs = self.activations
        first = np.dot(self.params.T, delta_l_plus_1)
        delta_l = first * self.activation_derivative_given_activation(inputs)
        #∂C/∂wljk=al−1k * δlj
        nudge = self.lr * np.outer(delta_l_plus_1, inputs) 
        self.params += nudge
        return delta_l[:-1]

In [33]:
def sigmoid(arr):
    return 1 / (1 + np.exp(-arr))        
        
def sigmoid_derivative_given_sigmoid_x(x):
    return (x) * (1 - (x))

In [44]:
class neural_network:
    
    def __init__(self, lr, dims, 
                 act_func = sigmoid, 
                 act_deriv_given_act = sigmoid_derivative_given_sigmoid_x, 
                 output_act = sigmoid, 
                 output_deriv = sigmoid_derivative_given_sigmoid_x
                 ):
        #self.lr = lr
        self.layers = []
        '''self.activation_function = act_func
        self.activation_derivative_given_activation =  act_deriv_given_act 
        self.output_activation = output_act
        self.output_derivative = output_deriv'''
        for i in range(len(dims) - 2):
            self.layers.append(perceptron_layer(dims[i], dims[i + 1], lr, act_func, act_deriv_given_act))
        self.layers.append(perceptron_layer(dims[-2], dims[-1], lr, output_act, output_deriv))
        pass
        
    def evaluate(self, inputs):
        activations = inputs
        for layer in self.layers:
            activations = layer.evaluate(activations)
            pass
        return activations
    
    def delta_last_layer(self, expected, predicted):
        return (expected - predicted) * sigmoid_derivative_given_sigmoid_x(predicted)
    
    def cost_function(self, output, predicted):
        return output - predicted
    
    def train_once(self, inputs, expected):
        predicted = nn.evaluate(inputs)
        cost = delta = self.delta_last_layer(expected, predicted)
        for layer in self.layers[::-1]:
            delta = layer.error_previous_layer(delta)
        return 0.5 * (cost ** 2).sum()
    
    def __repr__(self):
        return '\n'.join([repr(layer) for layer in self.layers])
    
    def train(self, epochs = 10000, data = []):
        error_rate = []
        weight_updation = []
        cost = 0
        for i in range(epochs):
            for ip, op in data:
                costx = self.train_once(ip, op)
                cost += costx
            error_rate.append([i, cost / epochs])
            weight_updation.append([i, self.layers[0].params[0][0]])
        cost /= epochs
        for ip, op in data:
            print(ip, nn.evaluate(ip))
        print(nn)
        print("Error =", cost)
        return error_rate, weight_updation
        

In [45]:
nn = neural_network(0.9, [3, 2, 1])

array([[0.68249437, 0.5138875 , 0.45738364, 0.82189228],
       [0.64959951, 0.22397992, 0.27148904, 0.75508619]])
array([[0.12000574, 0.08855163, 0.27712974]])


# 3 input XOR gate is being modelled
I shall hard code all the training data for now, since I have it typed out already for testing

In [46]:
data = np.array([[[0, 0, 0], [0]], [[0, 0, 1], [1]], [[0, 1, 0], [1]], [[0, 1, 1], [0]], [[1, 0, 0], [1]], [[1, 0, 1], [0]], [[1, 1, 0], [0]], [[1, 1, 1], [1]]], dtype=object)

In [48]:
first, second = nn.train(10000, data)

[0, 0, 0] [0.01238177]
[0, 0, 1] [0.992088]
[0, 1, 0] [0.99210669]
[0, 1, 1] [0.00800521]
[1, 0, 0] [0.99212681]
[1, 0, 1] [0.00802612]
[1, 1, 0] [0.00804672]
[1, 1, 1] [0.98691206]
array([[-10.87651335, -10.87433356, -10.8680778 ,  16.14175602],
       [ -1.50904396,  -1.50872706,  -1.50837415,   2.25305729]])
array([[ 24.69162854, -41.14854482,   8.16519376]])
Error = 4.319666687541849e-08
